In [1]:
import numpy as np

Define the end goal, that we want to achive in tensorflow:

In [2]:
np_pred = np.random.randn(4, 128)
np_true = np.array([[1],[1],[2],[3]])
print(np_pred.shape, np_true.shape)

(4, 128) (4, 1)


The goal is to comute the L2 distance between all permutations of the embeddings (rows of np_pred):

In [8]:
dist_pos = []
dist_neg = []
for i in range(np_pred.shape[0]-1):
    for j in range(i+1, np_pred.shape[0]):
        if np_true[i] == np_true[j]:
            dist_pos.append( np.linalg.norm(np_pred[i]-np_pred[j]) )
            print(f"Pos: {np.linalg.norm(np_pred[i]-np_pred[j])}")
        else:
            dist_neg.append( np.linalg.norm(np_pred[i]-np_pred[j]) )
            print(f"Neg: {np.linalg.norm(np_pred[i]-np_pred[j])}")

print(np.mean(dist_pos), np.mean(dist_neg))

Pos: 15.990136309634597
Neg: 16.223927757331612
Neg: 17.18952572066305
Neg: 15.66830997268424
Neg: 17.43500824522327
Neg: 14.929311302239624
15.990136309634597 16.28921659962836


Now lets go for a tensorflow implementation which will require posing this computation into a tensor opperation:

In [10]:
import tensorflow as tf

In [12]:
tf_pred = tf.constant(np_pred)
tf_true = tf.constant(np_true)
print(tf_pred.shape, tf_true.shape)

(4, 128) (4, 1)


In [14]:
tmp = tf.broadcast_to(tf_pred, (tf_pred.shape[0], tf_pred.shape[0], tf_pred.shape[1]))
tmp.shape

TensorShape([4, 4, 128])

In [16]:
difference_tensor = tmp - tf.transpose( tmp, [1,0,2] )
difference_tensor.shape

TensorShape([4, 4, 128])

In [36]:
pred_norm = tf.norm(difference_tensor, axis=2)

Nice! Now we need to get our maps for positive and negative examples:

In [57]:
tmp_true = tf.broadcast_to(tf_true, (tf_true.shape[0], tf_true.shape[0]))
true_diff =tf.cast( tmp_true == tf.transpose(tmp_true, [1,0]), tf.float64)
neg_diff = tf.cast( tmp_true != tf.transpose(tmp_true, [1,0]), tf.float64)

In [56]:
sum_pos = tf.reduce_sum(tf.math.multiply(true_diff, pred_norm))
count_pos = tf.reduce_sum(true_diff) - true_diff.shape[0] #Substract the counts from the diagonal
sum_pos/count_pos

<tf.Tensor: shape=(), dtype=float64, numpy=15.990136309634597>

In [59]:
sum_neg = tf.reduce_sum(tf.math.multiply(neg_diff, pred_norm))
count_neg = tf.reduce_sum(neg_diff)
sum_neg/count_neg

<tf.Tensor: shape=(), dtype=float64, numpy=16.28921659962836>

Nice ... slightly complicated, but thats tensorflow...